## 載入並處理資料

In [1]:
import chromadb
from datasets import load_dataset

dataset = load_dataset("squad", split="train")

ids = dataset["id"]
context = dataset["context"]
question = dataset["question"]
answers = dataset["answers"]

max_data_size = min(2500, len(dataset))
client = chromadb.Client()
collection = client.create_collection("squad")

collection.add(
    ids=["context-{}".format(i) for i in ids[:max_data_size]],
    documents=context[:max_data_size],
    metadatas=[{"from": "context"} for _ in range(max_data_size)],
)
collection.add(
    ids=["question-{}".format(i) for i in ids[:max_data_size]],
    documents=question[:max_data_size],
    metadatas=[{"from": "question"} for _ in range(max_data_size)],
)
collection.add(
    ids=["answers-{}".format(i) for i in ids[:max_data_size]],
    documents=[a["text"][0] for a in answers[:max_data_size]],
    metadatas=[{"from": "answers"} for _ in range(max_data_size)],
)

## 開始問答系統

In [2]:
query = "What is the oldest structure at Notre Dame?"
results = collection.query(
    query_texts=[query],
    n_results=1,
    where={"from": "answers"},)
answer = results["documents"][0][0]
print(answer)

Notre Dame
